In [22]:
import pandas as pd

# Step 1
Read the actual cases and convert the data from wide format to row format

In [23]:

try:
    files = []
    for i in range(3):
        files.append('confirmed_daily_cases{id}.csv'.format(id=i))
    data = pd.concat(map(pd.read_csv, files), ignore_index=True)
    print('found files for actual daily cases')
    print(data.head(4))
    confirmed_cases = data
except FileNotFoundError:
    import create_act_cases
    confirmed_cases = create_act_cases.confirmed_cases


found files for actual daily cases
   fips  act_cases        date                 county state date_last_week  \
0     0          0  2020-01-22  Statewide Unallocated    AL     2020-01-15   
1  1001          0  2020-01-22        Autauga County     AL     2020-01-15   
2  1003          0  2020-01-22        Baldwin County     AL     2020-01-15   
3  1005          0  2020-01-22        Barbour County     AL     2020-01-15   

   act_cases_last_week date_last_week.1 county_last_week  \
0                  NaN              NaN              NaN   
1                  NaN              NaN              NaN   
2                  NaN              NaN              NaN   
3                  NaN              NaN              NaN   

  date_last_week_last_week  weekly_cases  
0                      NaN           NaN  
1                      NaN           NaN  
2                      NaN           NaN  
3                      NaN           NaN  


## Step 1.1
Validate the data  

In [24]:
confirmed_cases[confirmed_cases.date == '2020-09-12'].groupby('date').sum()

,fips,act_cases,act_cases_last_week,weekly_cases
date,,,,
2020-09-12,95465426,6452813,6216459.0,236354.0


In [25]:
confirmed_cases

,fips,act_cases,date,county,state,date_last_week,act_cases_last_week,date_last_week.1,county_last_week,date_last_week_last_week,weekly_cases
0,0,0,2020-01-22,Statewide Unallocated,AL,2020-01-15,NaN,NaN,NaN,NaN,NaN
1,1001,0,2020-01-22,Autauga County,AL,2020-01-15,NaN,NaN,NaN,NaN,NaN
2,1003,0,2020-01-22,Baldwin County,AL,2020-01-15,NaN,NaN,NaN,NaN,NaN
3,1005,0,2020-01-22,Barbour County,AL,2020-01-15,NaN,NaN,NaN,NaN,NaN
4,1007,0,2020-01-22,Bibb County,AL,2020-01-15,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2094603,56037,7695,2021-11-07,Sweetwater County,WY,2021-10-31,7575.0,2021-10-31,Sweetwater County,2021-10-24,120.0
2094604,56039,5243,2021-11-07,Teton County,WY,2021-10-31,5204.0,2021-10-31,Teton County,2021-10-24,39.0
2094605,56041,3901,2021-11-07,Uinta County,WY,2021-10-31,3829.0,2021-10-31,Uinta County,2021-10-24,72.0
2094606,56043,1781,2021-11-07,Washakie County,WY,2021-10-31,1731.0,2021-10-31,Washakie County,2021-10-24,50.0


In [26]:
confirmed_cases['act_cases'] = confirmed_cases['weekly_cases']

confirmed_cases = confirmed_cases.drop(columns=['weekly_cases', 'date_last_week_last_week', 'county_last_week','date_last_week.1', 'act_cases_last_week', 'date_last_week'])

confirmed_cases

,fips,act_cases,date,county,state
0,0,NaN,2020-01-22,Statewide Unallocated,AL
1,1001,NaN,2020-01-22,Autauga County,AL
2,1003,NaN,2020-01-22,Baldwin County,AL
3,1005,NaN,2020-01-22,Barbour County,AL
4,1007,NaN,2020-01-22,Bibb County,AL
...,...,...,...,...,...
2094603,56037,120.0,2021-11-07,Sweetwater County,WY
2094604,56039,39.0,2021-11-07,Teton County,WY
2094605,56041,72.0,2021-11-07,Uinta County,WY
2094606,56043,50.0,2021-11-07,Washakie County,WY


# Step 2
Read the forcasting data from the multiple files

In [27]:
item = []
for i in range(6):
    item.append('fcast_data' + str(i+1) + '.csv')    

print(item)

data = pd.concat(map(pd.read_csv, item), ignore_index=True)

['fcast_data1.csv', 'fcast_data2.csv', 'fcast_data3.csv', 'fcast_data4.csv', 'fcast_data5.csv', 'fcast_data6.csv']


C:\Users\ChrisEvans\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
#data = data[data.target == "1 wk ahead inc case"]
#data = data[data.location_name != data.State]
data['date'] = pd.to_datetime(data['target_end_date'])

data = data.rename(columns={'point': "for_cases"})

data = data[['model','date','fips','for_cases','target']]

forcasted_cases = data

forcasted_cases.head(10)


,model,date,fips,for_cases,target
0,BPagano,2020-10-24,1,8924,1 wk ahead inc case
1,BPagano,2020-10-24,2,3554,1 wk ahead inc case
2,BPagano,2020-10-24,4,9092,1 wk ahead inc case
3,BPagano,2020-10-24,5,6166,1 wk ahead inc case
4,BPagano,2020-10-24,6,24521,1 wk ahead inc case
5,BPagano,2020-10-24,8,7660,1 wk ahead inc case
6,BPagano,2020-10-24,9,2029,1 wk ahead inc case
7,BPagano,2020-10-24,10,745,1 wk ahead inc case
8,BPagano,2020-10-24,11,421,1 wk ahead inc case
9,BPagano,2020-10-24,12,22717,1 wk ahead inc case


## Step 2.1
Read the population data

In [29]:
pop_data = pd.read_csv('covid_county_population_usafacts.csv')

pop_data = pop_data[['countyFIPS','population']]

pop_data = pop_data.rename(columns={'countyFIPS': "fips"})

pop_data.head(10)

,fips,population
0,0,0
1,1001,55869
2,1003,223234
3,1005,24686
4,1007,22394
5,1009,57826
6,1011,10101
7,1013,19448
8,1015,113605
9,1017,33254


# Step 3
Merge the two datasets together based on predication target date and the county

In [30]:
forcasted_cases['fips'] = forcasted_cases['fips'].astype(str)
forcasted_cases['date'] = forcasted_cases['date'].astype(str)

confirmed_cases['fips'] = confirmed_cases['fips'].astype(str)
confirmed_cases['date'] = confirmed_cases['date'].astype(str)

all_cases = pd.merge(forcasted_cases, confirmed_cases, on=['date','fips'], how='inner')

all_cases.head(10)

,model,date,fips,for_cases,target,act_cases,county,state
0,Columbia,2020-08-08,2013,0,1 wk ahead inc case,1.0,Aleutians East Borough,AK
1,Ensemble,2020-08-08,2013,0,1 wk ahead inc case,1.0,Aleutians East Borough,AK
2,Columbia,2020-08-08,2016,0,1 wk ahead inc case,-83.0,Aleutians West Census Area,AK
3,Ensemble,2020-08-08,2016,2,1 wk ahead inc case,-83.0,Aleutians West Census Area,AK
4,Columbia,2020-08-08,2020,745,1 wk ahead inc case,398.0,Municipality of Anchorage,AK
5,Ensemble,2020-08-08,2020,589,1 wk ahead inc case,398.0,Municipality of Anchorage,AK
6,LANL,2020-08-08,2020,589,1 wk ahead inc case,398.0,Municipality of Anchorage,AK
7,Columbia,2020-08-08,2050,0,1 wk ahead inc case,1.0,Bethel Census Area,AK
8,Ensemble,2020-08-08,2050,2,1 wk ahead inc case,1.0,Bethel Census Area,AK
9,LANL,2020-08-08,2050,2,1 wk ahead inc case,1.0,Bethel Census Area,AK


In [31]:
all_cases[all_cases.date == '2020-09-12'].groupby(['date','fips']).mean().groupby('date').sum()

,for_cases,act_cases
date,,
2020-09-12,274608.017081,234738.0


# Step 3.1 
Merge population data with all cases

In [32]:
pop_data['fips'] = pop_data['fips'].astype(str)

all_cases = pd.merge(all_cases, pop_data, on=['fips'], how='inner')
# add error calculations here 
all_cases.head(10)

,model,date,fips,for_cases,target,act_cases,county,state,population
0,Columbia,2020-08-08,2013,0,1 wk ahead inc case,1.0,Aleutians East Borough,AK,3337
1,Ensemble,2020-08-08,2013,0,1 wk ahead inc case,1.0,Aleutians East Borough,AK,3337
2,Columbia,2020-08-15,2013,0,2 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
3,Columbia,2020-08-15,2013,0,1 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
4,ESG,2020-08-15,2013,0,1 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
5,Ensemble,2020-08-15,2013,0,2 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
6,Ensemble,2020-08-15,2013,1,1 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
7,UGA-CEID,2020-08-15,2013,1,1 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
8,Columbia,2020-08-22,2013,0,3 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
9,Columbia,2020-08-22,2013,0,2 wk ahead inc case,0.0,Aleutians East Borough,AK,3337


Write the dataset to a file

In [33]:
all_cases.to_csv('all_cases.csv')

# Data Validation Test

In [34]:
all_cases[(all_cases.state == "MO") & (all_cases.county == "Jackson County ") & (all_cases.model == "Columbia") & (all_cases.target == "1 wk ahead inc case")].sort_values(by='date')

,model,date,fips,for_cases,target,act_cases,county,state,population
2331693,Columbia,2020-08-08,29095,1043,1 wk ahead inc case,1367.0,Jackson County,MO,703011
2331698,Columbia,2020-08-15,29095,370,1 wk ahead inc case,1241.0,Jackson County,MO,703011
2331709,Columbia,2020-08-22,29095,726,1 wk ahead inc case,1171.0,Jackson County,MO,703011
2331741,Columbia,2020-09-05,29095,340,1 wk ahead inc case,999.0,Jackson County,MO,703011
2331759,Columbia,2020-09-12,29095,638,1 wk ahead inc case,965.0,Jackson County,MO,703011
2331780,Columbia,2020-09-19,29095,416,1 wk ahead inc case,1236.0,Jackson County,MO,703011
2331805,Columbia,2020-09-26,29095,895,1 wk ahead inc case,1208.0,Jackson County,MO,703011
2331838,Columbia,2020-10-03,29095,292,1 wk ahead inc case,1252.0,Jackson County,MO,703011
2331874,Columbia,2020-10-10,29095,228,1 wk ahead inc case,1696.0,Jackson County,MO,703011
2331910,Columbia,2020-10-17,29095,655,1 wk ahead inc case,1514.0,Jackson County,MO,703011


In [35]:
all_cases[(all_cases.state == "AK") & (all_cases.county == "Aleutians East Borough ") & (all_cases.model == "Columbia") & (all_cases.target == "1 wk ahead inc case")].sort_values(by='date')


,model,date,fips,for_cases,target,act_cases,county,state,population
0,Columbia,2020-08-08,2013,0,1 wk ahead inc case,1.0,Aleutians East Borough,AK,3337
3,Columbia,2020-08-15,2013,0,1 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
10,Columbia,2020-08-22,2013,0,1 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
30,Columbia,2020-09-05,2013,0,1 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
42,Columbia,2020-09-12,2013,0,1 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
57,Columbia,2020-09-19,2013,0,1 wk ahead inc case,0.0,Aleutians East Borough,AK,3337
76,Columbia,2020-09-26,2013,0,1 wk ahead inc case,9.0,Aleutians East Borough,AK,3337
101,Columbia,2020-10-03,2013,0,1 wk ahead inc case,1.0,Aleutians East Borough,AK,3337
129,Columbia,2020-10-10,2013,0,1 wk ahead inc case,2.0,Aleutians East Borough,AK,3337
158,Columbia,2020-10-17,2013,0,1 wk ahead inc case,-9.0,Aleutians East Borough,AK,3337


In [36]:
all_cases[(all_cases.for_cases < 0)]

,model,date,fips,for_cases,target,act_cases,county,state,population
